In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [1]:
BASE_URL_1 = "https://kamis.kilimo.go.ke/site/market"
BASE_URL_2 = "https://kamis.kilimo.go.ke/site/market/{}"

ALL_ROWS = []
MAX_PRODUCTS = 273
PAGE_SIZE = 3000

def fetch_page(url):
    """Fetch URL and return BeautifulSoup object or None."""
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return BeautifulSoup(r.text, "html.parser")

def parse_table(soup):
    """Extract table rows from a KAMIS page."""
    table = soup.find("table")
    if not table:
        return []

    headers = [th.get_text(strip=True) for th in table.find_all("th")]
    rows = []

    for tr in table.find_all("tr")[1:]:
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        if len(cells) == len(headers):
            rows.append(dict(zip(headers, cells)))

    return rows

for product in range(1, MAX_PRODUCTS + 1):
    print(f"\n🔍 Scraping product {product}...")

    offset = 0
    while True:
        # First page uses BASE_URL_1, others use BASE_URL_2
        if offset == 0:
            url = f"{BASE_URL_1}?product={product}&per_page={PAGE_SIZE}"
        else:
            url = f"{BASE_URL_2.format(offset)}?product={product}&per_page={PAGE_SIZE}"

        print(f"  → Fetching: {url}")

        soup = fetch_page(url)
        if soup is None:
            print("    Failed request. Stopping product.")
            break

        rows = parse_table(soup)
        if not rows:
            print("    No more rows. Moving to next product.")
            break

        # Add product ID for reference
        for r in rows:
            r["ProductID"] = product

        ALL_ROWS.extend(rows)

        offset += PAGE_SIZE
        time.sleep(1)

# Save to CSV
df = pd.DataFrame(ALL_ROWS)
df.to_csv("kamis_data.csv", index=False)

print("\n✅ DONE! Saved to kamis_data.csv")


🔍 Scraping product 1...
  → Fetching: https://kamis.kilimo.go.ke/site/market?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/3000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/6000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/9000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/12000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/15000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/18000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/21000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/24000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/27000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/30000?product=1&per_page=3000
  → Fetching: https://kamis.kilimo.go.ke/site/market/33000?product=1&pe

In [2]:
pd.read_csv("kamis_data.csv")

,Commodity,Classification,Grade,Sex,Market,Wholesale,Retail,Supply Volume,County,Date,ProductID
0,Dry Maize,White Maize,-,-,Mwingi Town,46.67/Kg,55.00/Kg,40000.0,Kitui,2025-11-19,1
1,Dry Maize,White Maize,-,-,Kakamega Town,35.56/Kg,60.00/Kg,40000.0,Kakamega,2025-11-19,1
2,Dry Maize,White Maize,-,-,Kangeta,40.00/Kg,50.00/Kg,1300.0,Meru,2025-11-19,1
3,Dry Maize,Yellow Maize,-,-,Gikomba,77.78/Kg,100.00/Kg,NaN,Nairobi,2025-11-19,1
4,Dry Maize,White Maize,-,-,Gikomba,38.89/Kg,55.00/Kg,NaN,Nairobi,2025-11-19,1
...,...,...,...,...,...,...,...,...,...,...,...
1284887,Dragon Fruit,-,-,-,Mukuyu Market,-,500.00/Kg,NaN,Muranga,2025-11-19,273
1284888,Dragon Fruit,-,-,-,Voi Retail,-,150.00/Kg,20.0,Taita-Taveta,2025-11-11,273
1284889,Dragon Fruit,-,-,-,Mukuyu Market,-,500.00/Kg,NaN,Muranga,2025-08-13,273
1284890,Dragon Fruit,-,-,-,Voi Retail,-,150.00/Kg,50.0,Taita-Taveta,2025-07-18,273
